# Toy simulation model (previous example in Model Draft 8.2) 

In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import csv
import math

In [2]:
infect_rate = 0.17

## Start the first iteration here

In [3]:
empty_data_toy = { 'County name': ["A", "B", "C"], 
       "Total population": [100,100,100],
        "Total Commuters": [70,60,50],
       "Non-infected": [70,60,50],
        "Infected (Work)": [0,0,0],
       "Infected (Quarantine)": [0,0,0],
       "Already Infected": [0,0,0]}

SIR_df_toy = pd.DataFrame(empty_data_toy)

#SIR_df_toy.rename(index = {"0": "A",
                    # "1":"B", "2":"C"},)

SIR_df_toy.head()

,County name,Total population,Total Commuters,Non-infected,Infected (Work),Infected (Quarantine),Already Infected
0,A,100,70,70,0,0,0
1,B,100,60,60,0,0,0
2,C,100,50,50,0,0,0


In [4]:
empty_data_toy2 = { 'Home County': ["A","A","A", "B","B","B", "C","C","C"], 
       "Workplace County": ["A","B","C","A","B","C","A","B","C"],
        "Commuters": [20,25,25, 25,10,25,20,20,10 ]}

commuters_toy = pd.DataFrame(empty_data_toy2)

commuters_toy



,Home County,Workplace County,Commuters
0,A,A,20
1,A,B,25
2,A,C,25
3,B,A,25
4,B,B,10
5,B,C,25
6,C,A,20
7,C,B,20
8,C,C,10


In [79]:
### MISTAKE HERE, need to subtract the infected

In [5]:
## Suppose one person gets infected in A

counties = np.array(["A", "B", "C"])


Infected_county = "A" # Initial infection. Can pick randomly or specify

New_infected_countIES = pd.DataFrame(np.array(["A"]))

SIR_df_toy.at[0,"Infected (Work)"] = 1
SIR_df_toy

,County name,Total population,Total Commuters,Non-infected,Infected (Work),Infected (Quarantine),Already Infected
0,A,100,70,70,1,0,0
1,B,100,60,60,0,0,0
2,C,100,50,50,0,0,0


## Start the second iteration here

In [6]:
## And they go to work 


victim_countIES = pd.DataFrame(np.zeros(len(New_infected_countIES), dtype = int))

victim_countIES.insert(1, "Coming from", np.zeros(len(New_infected_countIES)))

for i in range(0, len(New_infected_countIES)):
    New_infected = New_infected_countIES.iat[i,0]
    temp = (pd.DataFrame(commuters_toy.loc[commuters_toy["Home County"].str.contains(New_infected)])).sample(axis = "rows")
    victim_countIES.iat[i,0] =temp.iat[0,1]
    victim_countIES.iat[i,1] =New_infected_countIES.iat[i,0]
    
victim_countIES

,0,Coming from
0,C,A


In [7]:
# Now we multiple A by the number of people in that county, WORKING IN THAT COUNTY
# Looking for number of commuters where new county = home county = C 

victim_workplaCES= pd.DataFrame(np.zeros(len(New_infected_countIES), dtype = int))
victim_workplaCES.insert(0, "Victim Counties", np.zeros(len(New_infected_countIES)))

## Finding the common array 
for i in range(0, len(New_infected_countIES)): 
    print(np.where(commuters_toy["Home County"].str.contains(victim_countIES.iat[i,0])))
    print(np.where(commuters_toy["Workplace County"].str.contains(victim_countIES.iat[i,0])))
    victim_workplaCES.iat[i,0] = victim_countIES.iat[i,0]
    victim_workplaCES.iat[i,1] = np.intersect1d(np.where(commuters_toy["Home County"].str.contains(victim_countIES.iat[i,0])),np.where(commuters_toy["Workplace County"].str.contains(victim_countIES.iat[i,0])))
victim_workplaCES

(array([6, 7, 8]),)
(array([2, 5, 8]),)


,Victim Counties,0
0,C,8


In [8]:
new_infected_at_workplace = pd.DataFrame(np.zeros(len(SIR_df_toy), dtype = int))#SIR_df_toy[["Infected (Work)"]]
new_infected_at_workplace.insert(0, "Victim Counties", ["A", "B", "C"])
new_infected_at_workplace.rename(columns = {0:'Newly Infected'}, inplace = True)
new_infected_at_workplace

,Victim Counties,Newly Infected
0,A,0
1,B,0
2,C,0


## Scratch work for below but there's an error! 

In [9]:

current_county = new_infected_at_workplace.at[2,"Victim Counties"] ## CHANGE TO I
workplace_people = commuters_toy.at[int(victim_workplaCES.loc[victim_workplaCES["Victim Counties"]==current_county,0]),"Commuters"]

infection_origin = victim_countIES.loc[victim_countIES[0]==current_county,"Coming from"][0]

#infectors = SIR_df_toy

infectors = SIR_df_toy.loc[SIR_df_toy["County name"]==infection_origin,"Infected (Work)"][0]

# This is an issue for later 
#subset = commuters_toy.loc[commuters_toy["Home County"]==infection_origin 
#subset.loc[commuters_toy["Workplace County"]==current_county, "Commuters"]

In [98]:
#New Infected people = Workplace_CC * Infectionrate * Infected_A


for i in range(0, len(SIR_df_toy)):
    if (new_infected_at_workplace.at[i,"Victim Counties"] in victim_workplaCES[["Victim Counties"]].values): 
        current_county = new_infected_at_workplace.at[i,"Victim Counties"]
        workplace_people = commuters_toy.at[int(victim_workplaCES.loc[victim_workplaCES["Victim Counties"]==current_county,0]),"Commuters"]
        infection_origin = victim_countIES.loc[victim_countIES[0]==current_county,"Coming from"][0]
        infectors = SIR_df_toy.loc[SIR_df_toy["County name"]==infection_origin,"Infected (Work)"][0]
        new_infected_at_workplace.at[i,"Newly Infected"] = round(workplace_people * infect_rate * infectors)
        
        change_commuters = np.intersect1d(np.where(commuters_toy["Home County"].str.contains(infection_origin)),np.where(commuters_toy["Workplace County"].str.contains(current_county)))
        commuters_toy.at[int(change_commuters),"Commuters"] = commuters_toy.at[int(change_commuters),"Commuters"] - infectors
        
new_infected_at_workplace



TypeError: cannot convert the series to <class 'int'>

In [99]:
## Update data sets 

## Infected(Work) gets moved to Infected (Quarantine)
## Total commuters gets subtracted 
## commuting data set gets updated 

## Noninfected gets moved to Infected(work)



for i in range(0, len(new_infected_at_workplace)):
    move_infected = SIR_df_toy.at[i,"Infected (Work)"]
    
    SIR_df_toy.at[i,"Infected (Quarantine)"] = SIR_df_toy.at[i,"Infected (Quarantine)"] + move_infected #1
    SIR_df_toy.at[i,"Total Commuters"] = SIR_df_toy.at[i,"Total Commuters"] - move_infected #2
    SIR_df_toy.at[i,"Non-infected"] =  SIR_df_toy.at[i,"Non-infected"] - move_infected #3
    SIR_df_toy.at[i,"Infected (Work)"] = SIR_df_toy.at[i,"Infected (Work)"] - move_infected + new_infected_at_workplace.at[i,"Newly Infected"] # 4
    
    SIR_df_toy.at[i,"Non-infected"] = SIR_df_toy.at[i,"Non-infected"] - new_infected_at_workplace.at[i,"Newly Infected"] # 5
    


In [10]:
commuters_toy

,Home County,Workplace County,Commuters
0,A,A,20
1,A,B,25
2,A,C,25
3,B,A,25
4,B,B,10
5,B,C,25
6,C,A,20
7,C,B,20
8,C,C,10


In [11]:
SIR_df_toy

,County name,Total population,Total Commuters,Non-infected,Infected (Work),Infected (Quarantine),Already Infected
0,A,100,70,70,1,0,0
1,B,100,60,60,0,0,0
2,C,100,50,50,0,0,0


In [12]:
New_infected_countIES = new_infected_at_workplace


In [13]:
New_infected_countIES

,Victim Counties,Newly Infected
0,A,0
1,B,0
2,C,0
